In [10]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split

In [11]:
dataset_path = '/kaggle/input/miv2024/LegumesEtFruitsDataset'

input_size = (102, 102) 

++++++++++++++++++++++ = ImageDataGenerator(rescale=1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    validation_split=0.3)

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=input_size,
    batch_size=26,
    class_mode='categorical', # categorical for multi-class classification
    subset='training')


val_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.15)


val_generator = val_datagen.flow_from_directory(
    dataset_path,
    target_size=input_size,
    batch_size=26,
    class_mode='categorical',
    subset='validation')

test_datagen = ImageDataGenerator(rescale=1./255,
                                  validation_split=0.15)


test_generator = test_datagen.flow_from_directory(
    dataset_path,
    target_size=input_size,
    batch_size=26,
    class_mode='categorical',
    subset='validation')

Found 843 images belonging to 14 classes.
Found 172 images belonging to 14 classes.
Found 172 images belonging to 14 classes.


In [12]:
# get the number of classes
num_classes = train_generator.num_classes
# get class labels
class_labels = list(train_generator.class_indices.keys())

In [13]:
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(input_size[0], input_size[1], 3)),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(256, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Conv2D(512, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    
    keras.layers.Flatten(),
    
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    
    keras.layers.Dense(num_classes, activation='softmax')
])

In [14]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 100, 100, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 50, 50, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 48, 48, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 24, 24, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 22, 22, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 11, 11, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 9, 9, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 2, 2, 512)      │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 14)             │         7,182 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,838,414 (7.01 MB)

 Trainable params: 1,838,414 (7.01 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
print("Num GPUs Available: ", tf.config.list_physical_devices('GPU'))

Num GPUs Available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [16]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

#early_stopping = EarlyStopping(monitor='val_loss', patience=5)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True)

with tf.device('/device:GPU:0'):
    history = model.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[model_checkpoint]
    )

Epoch 1/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.0663 - loss: 2.6610 - val_accuracy: 0.1047 - val_loss: 2.5892
Epoch 2/30


W0000 00:00:1716438003.147535     128 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


33/33 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.1517 - loss: 2.4868 - val_accuracy: 0.1802 - val_loss: 2.1892
Epoch 3/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.2492 - loss: 2.1524 - val_accuracy: 0.2326 - val_loss: 1.9555
Epoch 4/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.2734 - loss: 2.0628 - val_accuracy: 0.3372 - val_loss: 2.0312
Epoch 5/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2723 - loss: 2.2083 - val_accuracy: 0.3198 - val_loss: 1.9366
Epoch 6/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.2662 - loss: 2.1664 - val_accuracy: 0.3779 - val_loss: 1.6763
Epoch 7/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.3969 - loss: 1.6766 - val_accuracy: 0.4709 - val_loss: 1.4900
Epoch 8/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 51s 1s/step - accuracy: 0.4493 - loss: 1.5178 - val_accuracy: 0.5407 - val_loss: 1.3936
Epoch 9/30
33/33 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step - accuracy: 0.4851 - loss: 1.4634 - val_accuracy: 0.3837 - val_loss: 3.4331
Epo

In [17]:
from tensorflow.keras.models import load_model

best_model = load_model('best_model.keras')
val_loss, val_accuracy = best_model.evaluate(val_generator)
print(f"Validation accuracy of the best model: {val_accuracy * 100:.2f}%")
test_loss, test_accuracy = best_model.evaluate(test_generator)
print(f"Testing accuracy of the best model: {test_accuracy * 100:.2f}%")

7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 296ms/step - accuracy: 0.8828 - loss: 0.5173


W0000 00:00:1716439477.739996     126 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Validation accuracy of the best model: 90.70%
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 279ms/step - accuracy: 0.8977 - loss: 0.5066
Testing accuracy of the best model: 90.70%
